In [ ]:
import torch
import os
from torch import nn
from time import time
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import utils
from unet_model import ConvAE, ConvAE_M, ConvAE_M_Loc, ConvAE_M_shallow, UNet, UNet_M, UNet_M_Loc, UNet_M_Loc_strided, UNet_M_Loc_deepNet, UNet_M_LC, WNet, WNet2
import torchvision
import random 

from models import Linear, Linear_M, Linear_M_Loc, LinearAE, LinearAE_M, LinearAE_M_Loc

font = {'size'   : 16}

matplotlib.rc('font', **font)

freq = ['400MHz','900MHz','1pt5GHz','2pt4GHz'][0]

In [ ]:
# Inputs : train
f_f1        = os.path.join('data', 'E_field_COMSOL', freq , 'data_f1_04_' + freq + '.pt')
f_f1_05     = os.path.join('data', 'E_field_COMSOL', freq , 'data_f1_05_' + freq + '.pt')
f_f1_06     = os.path.join('data', 'E_field_COMSOL', freq , 'data_f1_06_' + freq + '.pt')
f_f1_18     = os.path.join('data', 'E_field_COMSOL', freq , 'data_f1_18_' + freq + '.pt')
f_f1_20     = os.path.join('data', 'E_field_COMSOL', freq , 'data_f1_20_' + freq + '.pt')
f_f1_38     = os.path.join('data', 'E_field_COMSOL', freq , 'data_f1_38_' + freq + '.pt')
f_f1_41     = os.path.join('data', 'E_field_COMSOL', freq , 'data_f1_41_' + freq + '.pt')
f_f1_42     = os.path.join('data', 'E_field_COMSOL', freq , 'data_f1_42_' + freq + '.pt')
f_f1_43     = os.path.join('data', 'E_field_COMSOL', freq , 'data_f1_43_' + freq + '.pt')
f_f1_44     = os.path.join('data', 'E_field_COMSOL', freq , 'data_f1_44_' + freq + '.pt')
f_f1_45     = os.path.join('data', 'E_field_COMSOL', freq , 'data_f1_45_' + freq + '.pt')
f_f1_46     = os.path.join('data', 'E_field_COMSOL', freq , 'data_f1_46_' + freq + '.pt')
f_f1_47     = os.path.join('data', 'E_field_COMSOL', freq , 'data_f1_47_' + freq + '.pt')
f_f1_48     = os.path.join('data', 'E_field_COMSOL', freq , 'data_f1_48_' + freq + '.pt')
f_f1_49     = os.path.join('data', 'E_field_COMSOL', freq , 'data_f1_49_' + freq + '.pt')
# inputs: val
f_f1_50     = os.path.join('data', 'E_field_COMSOL', freq , 'data_f1_50_' + freq + '.pt')
f_f1_51     = os.path.join('data', 'E_field_COMSOL', freq , 'data_f1_51_' + freq + '.pt')
f_f1_52     = os.path.join('data', 'E_field_COMSOL', freq , 'data_f1_52_' + freq + '.pt')
f_f1_53     = os.path.join('data', 'E_field_COMSOL', freq , 'data_f1_53_' + freq + '.pt')
f_f1_54     = os.path.join('data', 'E_field_COMSOL', freq , 'data_f1_54_' + freq + '.pt')

# loc: train + test
f_loc    = os.path.join('data', 'E_field_COMSOL', freq, 'f_loc_outside.pt')

# MRI
f_MRI    = os.path.join('data', 'E_field_COMSOL', freq, 'MRI_04.pt')
f_MRI_05    = os.path.join('data', 'E_field_COMSOL', freq, 'MRI_05.pt')
f_MRI_06    = os.path.join('data', 'E_field_COMSOL', freq, 'MRI_06.pt')
f_MRI_18    = os.path.join('data', 'E_field_COMSOL', freq, 'MRI_18.pt')
f_MRI_20    = os.path.join('data', 'E_field_COMSOL', freq, 'MRI_20.pt')
f_MRI_38    = os.path.join('data', 'E_field_COMSOL', freq, 'MRI_38.pt')
f_MRI_41    = os.path.join('data', 'E_field_COMSOL', freq, 'MRI_41.pt')
f_MRI_42    = os.path.join('data', 'E_field_COMSOL', freq, 'MRI_42.pt')
f_MRI_43    = os.path.join('data', 'E_field_COMSOL', freq, 'MRI_43.pt')
f_MRI_44    = os.path.join('data', 'E_field_COMSOL', freq, 'MRI_44.pt')
f_MRI_45    = os.path.join('data', 'E_field_COMSOL', freq, 'MRI_45.pt')
f_MRI_46    = os.path.join('data', 'E_field_COMSOL', freq, 'MRI_46.pt')
f_MRI_47    = os.path.join('data', 'E_field_COMSOL', freq, 'MRI_47.pt')
f_MRI_48    = os.path.join('data', 'E_field_COMSOL', freq, 'MRI_48.pt')
f_MRI_49    = os.path.join('data', 'E_field_COMSOL', freq, 'MRI_49.pt')
f_MRI_50    = os.path.join('data', 'E_field_COMSOL', freq, 'MRI_50.pt')
f_MRI_51    = os.path.join('data', 'E_field_COMSOL', freq, 'MRI_51.pt')
f_MRI_52    = os.path.join('data', 'E_field_COMSOL', freq, 'MRI_52.pt')
f_MRI_53    = os.path.join('data', 'E_field_COMSOL', freq, 'MRI_53.pt')
f_MRI_54    = os.path.join('data', 'E_field_COMSOL', freq, 'MRI_54.pt')


n_batch = 32
n_workers = 0
n_subsample = 3 

total_train_patients = 14
impulse_mag = 1
MRI_loc_impulse_msg = 10 # for setting the size of the antenna impulse 

# choosing seed randomly from(0, 500)
seed_n = 500*torch.rand(1)
seed = int(seed_n)


orig_size = [434,362]
orig_size_down = [int(a/n_subsample) for a in orig_size] 

data_train = 'f1' #list picks which frequency to use
#device = torch.device('cuda')
device = torch.device('cpu')

model_type = ['Linear', 'Linear_M', 'Linear_M_Loc', 
              'LinearAE', 'LinearAE_M', 'LinearAE_M_Loc',
              'ConvAE', 'ConvAE_M', 'ConvAE_M_Loc',
              'unet', 'unet_M', 'UNet_M_LC', 'UNet_M_Loc', 'UNet_M_Loc_strided', 'UNet_M_Loc_deepNet',
              'wnet', 'wnet2'][13] #list picks which model to use
print(model_type)


In [ ]:
utils.set_seed(seed)


f_log = f'Hyper_param_'+str(model_type)+ '_'+ freq + '_D' + str(n_subsample) + '.txt'
f_subfolder = model_type + '_D' + str(n_subsample) + '_' + data_train 


f_output_images = os.path.join('output', freq,  'images', model_type)
f_model_output = os.path.join('output', freq, 'models')

# make output folders
os.makedirs(os.path.join(f_output_images, f_subfolder), exist_ok=True)
os.makedirs(os.path.join(f_model_output, f_subfolder), exist_ok=True)

f_sub_sub_folder = f'seed: {seed}'
os.makedirs(os.path.join(f_output_images, f_subfolder, f_sub_sub_folder), exist_ok=True)


In [ ]:
def complex_resize(x, shape):
    x_real = x.real.clone()
    x_imag = x.imag.clone()
    x_real_resize = torchvision.transforms.Resize(shape, antialias=True)(x_real)
    x_imag_resize = torchvision.transforms.Resize(shape, antialias=True)(x_imag)    
    y = x_real_resize + 1j* x_imag_resize
    return y

In [ ]:
# load  train data + val data
fdata = torch.load(f_f1).to(device)
fdata_05 = torch.load(f_f1_05).to(device)
fdata_06 = torch.load(f_f1_06).to(device)
fdata_18 = torch.load(f_f1_18).to(device)
fdata_20 = torch.load(f_f1_20).to(device)
fdata_38 = torch.load(f_f1_38).to(device)
fdata_41 = torch.load(f_f1_41).to(device)
fdata_42 = torch.load(f_f1_42).to(device)
fdata_43 = torch.load(f_f1_43).to(device)
fdata_44 = torch.load(f_f1_44).to(device)
fdata_45 = torch.load(f_f1_45).to(device)
fdata_46 = torch.load(f_f1_46).to(device)
fdata_47 = torch.load(f_f1_47).to(device)
fdata_48 = torch.load(f_f1_48).to(device)
fdata_49 = torch.load(f_f1_49).to(device)
fdata_50 = torch.load(f_f1_50).to(device)
fdata_51 = torch.load(f_f1_51).to(device)
fdata_52 = torch.load(f_f1_52).to(device)
fdata_53 = torch.load(f_f1_53).to(device)
fdata_54 = torch.load(f_f1_54).to(device)

# Load loc data
loc = torch.load(f_loc).to(device)

# load the MRI
MRI = torch.load(f_MRI).to(device)
MRI_05 = torch.load(f_MRI_05).to(device)
MRI_06 = torch.load(f_MRI_06).to(device)
MRI_18 = torch.load(f_MRI_18).to(device)
MRI_20 = torch.load(f_MRI_20).to(device)
MRI_38 = torch.load(f_MRI_38).to(device)
MRI_41 = torch.load(f_MRI_41).to(device)
MRI_42 = torch.load(f_MRI_42).to(device)
MRI_43 = torch.load(f_MRI_43).to(device)
MRI_44 = torch.load(f_MRI_44).to(device)
MRI_45 = torch.load(f_MRI_45).to(device)
MRI_46 = torch.load(f_MRI_46).to(device)
MRI_47 = torch.load(f_MRI_47).to(device)
MRI_48 = torch.load(f_MRI_48).to(device)
MRI_49 = torch.load(f_MRI_49).to(device)
MRI_50 = torch.load(f_MRI_50).to(device)
MRI_51 = torch.load(f_MRI_51).to(device)
MRI_52 = torch.load(f_MRI_52).to(device)
MRI_53 = torch.load(f_MRI_53).to(device)
MRI_54 = torch.load(f_MRI_54).to(device)

In [ ]:
# padding parameters: 
tot_x_axis_len = 370+30 
tot_y_axis_len = 444+30 

# apply pad to location
loc_new = utils.apply_pad_loc(loc, tot_x_axis_len, tot_y_axis_len)


In [ ]:
# pad E-field 
fdata_pad = utils.complex_padding(fdata, tot_y_axis_len, tot_x_axis_len)
fdata_05_pad = utils.complex_padding(fdata_05, tot_y_axis_len, tot_x_axis_len)
fdata_06_pad = utils.complex_padding(fdata_06, tot_y_axis_len, tot_x_axis_len)
fdata_18_pad = utils.complex_padding(fdata_18, tot_y_axis_len, tot_x_axis_len)
fdata_20_pad = utils.complex_padding(fdata_20, tot_y_axis_len, tot_x_axis_len)
fdata_38_pad = utils.complex_padding(fdata_38, tot_y_axis_len, tot_x_axis_len)
fdata_41_pad = utils.complex_padding(fdata_41, tot_y_axis_len, tot_x_axis_len)
fdata_42_pad = utils.complex_padding(fdata_42, tot_y_axis_len, tot_x_axis_len)
fdata_43_pad = utils.complex_padding(fdata_43, tot_y_axis_len, tot_x_axis_len)
fdata_44_pad = utils.complex_padding(fdata_44, tot_y_axis_len, tot_x_axis_len)
fdata_45_pad = utils.complex_padding(fdata_45, tot_y_axis_len, tot_x_axis_len)
fdata_46_pad = utils.complex_padding(fdata_46, tot_y_axis_len, tot_x_axis_len)
fdata_47_pad = utils.complex_padding(fdata_47, tot_y_axis_len, tot_x_axis_len)
fdata_48_pad = utils.complex_padding(fdata_48, tot_y_axis_len, tot_x_axis_len)
fdata_49_pad = utils.complex_padding(fdata_49, tot_y_axis_len, tot_x_axis_len)
fdata_50_pad = utils.complex_padding(fdata_50, tot_y_axis_len, tot_x_axis_len)
fdata_51_pad = utils.complex_padding(fdata_51, tot_y_axis_len, tot_x_axis_len)
fdata_52_pad = utils.complex_padding(fdata_52, tot_y_axis_len, tot_x_axis_len)
fdata_53_pad = utils.complex_padding(fdata_53, tot_y_axis_len, tot_x_axis_len)
fdata_54_pad = utils.complex_padding(fdata_54, tot_y_axis_len, tot_x_axis_len)

# pad MRI
MRI_pad = utils.apply_pad(MRI, tot_x_axis_len, tot_y_axis_len)
MRI_05_pad = utils.apply_pad(MRI_05, tot_x_axis_len, tot_y_axis_len)
MRI_06_pad = utils.apply_pad(MRI_06, tot_x_axis_len, tot_y_axis_len)
MRI_18_pad = utils.apply_pad(MRI_18, tot_x_axis_len, tot_y_axis_len)
MRI_20_pad = utils.apply_pad(MRI_20, tot_x_axis_len, tot_y_axis_len)
MRI_38_pad = utils.apply_pad(MRI_38, tot_x_axis_len, tot_y_axis_len)
MRI_41_pad = utils.apply_pad(MRI_41, tot_x_axis_len, tot_y_axis_len)
MRI_42_pad = utils.apply_pad(MRI_42, tot_x_axis_len, tot_y_axis_len)
MRI_43_pad = utils.apply_pad(MRI_43, tot_x_axis_len, tot_y_axis_len)
MRI_44_pad = utils.apply_pad(MRI_44, tot_x_axis_len, tot_y_axis_len)
MRI_45_pad = utils.apply_pad(MRI_45, tot_x_axis_len, tot_y_axis_len)
MRI_46_pad = utils.apply_pad(MRI_46, tot_x_axis_len, tot_y_axis_len)
MRI_47_pad = utils.apply_pad(MRI_47, tot_x_axis_len, tot_y_axis_len)
MRI_48_pad = utils.apply_pad(MRI_48, tot_x_axis_len, tot_y_axis_len)
MRI_49_pad = utils.apply_pad(MRI_49, tot_x_axis_len, tot_y_axis_len)
MRI_50_pad = utils.apply_pad(MRI_50, tot_x_axis_len, tot_y_axis_len)
MRI_51_pad = utils.apply_pad(MRI_51, tot_x_axis_len, tot_y_axis_len)
MRI_52_pad = utils.apply_pad(MRI_52, tot_x_axis_len, tot_y_axis_len)
MRI_53_pad = utils.apply_pad(MRI_53, tot_x_axis_len, tot_y_axis_len)
MRI_54_pad = utils.apply_pad(MRI_54, tot_x_axis_len, tot_y_axis_len)


In [ ]:
# downsampling data
new_size = [int(a/n_subsample) for a in [tot_y_axis_len, tot_x_axis_len]] # FA: my code was here
fdata = complex_resize(fdata_pad, new_size)
fdata_05 = complex_resize(fdata_05_pad, new_size)
fdata_06 = complex_resize(fdata_06_pad, new_size)
fdata_18 = complex_resize(fdata_18_pad, new_size)
fdata_20 = complex_resize(fdata_20_pad, new_size)
fdata_38 = complex_resize(fdata_38_pad, new_size)
fdata_41 = complex_resize(fdata_41_pad, new_size)
fdata_42 = complex_resize(fdata_42_pad, new_size)
fdata_43 = complex_resize(fdata_43_pad, new_size)
fdata_44 = complex_resize(fdata_44_pad, new_size)
fdata_45 = complex_resize(fdata_45_pad, new_size)
fdata_46 = complex_resize(fdata_46_pad, new_size)
fdata_47 = complex_resize(fdata_47_pad, new_size)
fdata_48 = complex_resize(fdata_48_pad, new_size)
fdata_49 = complex_resize(fdata_49_pad, new_size)
fdata_50 = complex_resize(fdata_50_pad, new_size)
fdata_51 = complex_resize(fdata_51_pad, new_size)
fdata_52 = complex_resize(fdata_52_pad, new_size)
fdata_53 = complex_resize(fdata_53_pad, new_size)
fdata_54 = complex_resize(fdata_54_pad, new_size)

# downsampling location
loc = (loc_new/n_subsample).round().int()

# downsampling MRI 
MRI    = torchvision.transforms.Resize(new_size, antialias=True)(MRI_pad.unsqueeze(0)).squeeze()
MRI_05 = torchvision.transforms.Resize(new_size, antialias=True)(MRI_05_pad.unsqueeze(0)).squeeze()
MRI_06 = torchvision.transforms.Resize(new_size, antialias=True)(MRI_06_pad.unsqueeze(0)).squeeze()
MRI_18 = torchvision.transforms.Resize(new_size, antialias=True)(MRI_18_pad.unsqueeze(0)).squeeze()
MRI_20 = torchvision.transforms.Resize(new_size, antialias=True)(MRI_20_pad.unsqueeze(0)).squeeze()
MRI_38 = torchvision.transforms.Resize(new_size, antialias=True)(MRI_38_pad.unsqueeze(0)).squeeze()
MRI_41 = torchvision.transforms.Resize(new_size, antialias=True)(MRI_41_pad.unsqueeze(0)).squeeze()
MRI_42 = torchvision.transforms.Resize(new_size, antialias=True)(MRI_42_pad.unsqueeze(0)).squeeze()
MRI_43 = torchvision.transforms.Resize(new_size, antialias=True)(MRI_43_pad.unsqueeze(0)).squeeze()
MRI_44 = torchvision.transforms.Resize(new_size, antialias=True)(MRI_44_pad.unsqueeze(0)).squeeze()
MRI_45 = torchvision.transforms.Resize(new_size, antialias=True)(MRI_45_pad.unsqueeze(0)).squeeze()
MRI_46 = torchvision.transforms.Resize(new_size, antialias=True)(MRI_46_pad.unsqueeze(0)).squeeze()
MRI_47 = torchvision.transforms.Resize(new_size, antialias=True)(MRI_47_pad.unsqueeze(0)).squeeze()
MRI_48 = torchvision.transforms.Resize(new_size, antialias=True)(MRI_48_pad.unsqueeze(0)).squeeze()
MRI_49 = torchvision.transforms.Resize(new_size, antialias=True)(MRI_49_pad.unsqueeze(0)).squeeze()
MRI_50 = torchvision.transforms.Resize(new_size, antialias=True)(MRI_50_pad.unsqueeze(0)).squeeze()
MRI_51 = torchvision.transforms.Resize(new_size, antialias=True)(MRI_51_pad.unsqueeze(0)).squeeze()
MRI_52 = torchvision.transforms.Resize(new_size, antialias=True)(MRI_52_pad.unsqueeze(0)).squeeze()
MRI_53 = torchvision.transforms.Resize(new_size, antialias=True)(MRI_53_pad.unsqueeze(0)).squeeze()
MRI_54 = torchvision.transforms.Resize(new_size, antialias=True)(MRI_54_pad.unsqueeze(0)).squeeze()


In [ ]:
data_shape = fdata.shape
print(data_shape)

if model_type in ['LinearAE', 'LinearAE_M', 'ConvAE', 'ConvAE_M', 'unet',  'unet_M', 'UNet_M_LC' , 'wnet', 'wnet2']:
    #Training data
    x = torch.zeros(data_shape).to(device)
    for i in range(loc.shape[0]):
        impulse_input = torch.zeros(data_shape[1:])
        r = loc[i, 1]
        c = loc[i, 0]
        impulse_input[r,c] = impulse_mag 
        x[i,:,:] = impulse_input
elif model_type in ['Linear_M_Loc', 'LinearAE_M_Loc', 'ConvAE_M_Loc', 'UNet_M_Loc', 'UNet_M_Loc_strided', 'UNet_M_Loc_deepNet']:
    #Training data
    x = torch.zeros(data_shape).to(device)
    for i in range(loc.shape[0]):
        impulse_input = torch.zeros(data_shape[1:])
        r = loc[i, 1]
        c = loc[i, 0]
        impulse_input[r,c] = MRI_loc_impulse_msg 
        x[i,:,:] = impulse_input
elif model_type in ['Linear', 'Linear_M']: 
    x = loc.float().to(device)
else:
    raise Exception('Unknown model type')
    

In [ ]:
# final dataset concatenate: across patients
x_train = torch.cat([x for i in range(14)], dim = 0)
x_train_MRI = torch.cat([temp.unsqueeze(1).tile(x.shape[0], (1)).permute(1,0,2) 
                         for temp in [MRI, MRI_05, MRI_06, MRI_18, MRI_20, MRI_38, MRI_41, MRI_42, MRI_43, MRI_44, MRI_45, MRI_46, MRI_47, MRI_48]], dim = 0)
fdata_train = torch.cat([fdata, fdata_05, fdata_06, fdata_18, fdata_20, fdata_38, fdata_41, fdata_42, fdata_43, fdata_44, fdata_45, fdata_46, fdata_47, fdata_48], dim = 0)

x_val = torch.cat([x for i in range(6)], dim = 0)
x_val_MRI = torch.cat([temp.unsqueeze(1).tile(x.shape[0], (1)).permute(1,0,2) 
                         for temp in [MRI_49, MRI_50, MRI_51, MRI_52, MRI_53, MRI_54]], dim = 0)
fdata_val = torch.cat([fdata_49, fdata_50, fdata_51, fdata_52, fdata_53, fdata_54], dim = 0)
print(x_val_MRI.shape)

In [ ]:
fdata_train_masked = fdata_train.cpu() * (x_train_MRI.cpu()>0)

data_max_train = torch.max(fdata_train_masked.abs()).item()
data_min_train = torch.min(fdata_train_masked.abs()).item()
data_mean_train = torch.mean(fdata_train_masked).item()
data_std_train = torch.std(fdata_train_masked.abs()).item()
#print(f'max = {data_max_train},\nmin = {data_min_train},\nmean = {data_mean_train},\nstd = {data_std_train}')

fdata_val_masked = fdata_val.cpu() * (x_val_MRI.cpu()>0)
data_max_val = torch.max(fdata_val_masked.abs()).item()
data_min_val = torch.min(fdata_val_masked.abs()).item()
data_mean_val = torch.mean(fdata_val_masked).item()
data_std_val = torch.std(fdata_val_masked.abs()).item()
#print(f'\nmax = {data_max_val},\nmin = {data_min_val},\nmean = {data_mean_val},\nstd = {data_std_val}')


In [ ]:
class FA_Dataset(torch.utils.data.Dataset):
    """My custom dataset."""

    def __init__(self, loc, MRI, fdata, orig_size_down=orig_size_down, transformF=None, transformMRI=None):
        """
        Arguments:
            loc (string): location vector. Should be N x 2
            MRI (string): MRI for each location. Should be N x W x H
            fdata (callable): Impulse response. Should be N x W x H
        """
        self.loc = loc
        self.MRI = MRI
        self.masks = (MRI > 0).float()
        self.fdata = fdata
        self.transformF = transformF
        self.transformMRI = transformMRI
        self.orig_size_down = orig_size_down

    def __len__(self):
        return self.MRI.shape[0]

    def __getitem__(self, idx):
        return [self.loc[idx], self.transformMRI(self.MRI[idx]), self.masks[idx]], self.transformF(self.fdata[idx])

In [ ]:
# data_shape = fdata.shape
def transG(m, s):
    return lambda x: (x - m)/s
transF_train = transG(data_mean_train,data_std_train)
transF_val   = transG(data_mean_val,data_std_val)
transMRI = lambda x: (x.float()/255)*2-1

# dataset
trainset = FA_Dataset(x_train, x_train_MRI, fdata_train, transformF=transF_train, transformMRI=transMRI)
testset  = FA_Dataset(x_val, x_val_MRI, fdata_val, transformF=transF_val, transformMRI=transMRI)

# dataloader
trainloader = torch.utils.data.DataLoader(trainset, batch_size=n_batch, shuffle=True, num_workers=n_workers)# ask lorenxo
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=True, num_workers=n_workers)# ask lorenxo

#Data properties
d = torch.prod(torch.Tensor(list(data_shape[1:]))).int().item()# this is fine bcz outputs are still 434x362 images

In [ ]:
#Model hyperparameters that are the same for all models
criterion = lambda x,y: ( (x-y).abs()**2 ).mean() #MSE

momentum = 0.9
weight_decay = 5e-4
n_bottleneck = 64

# defining new hyperparameter for random selection: (1e-7, 1e-2)
rand_lr = 1+ 3*torch.rand(1) 
lr =  10**-rand_lr.item()
print(lr)   

# choose lernels in range (3, 12)
rand_kernels = 3 + 9*torch.rand(1)
n_kernel = int(rand_kernels)

if n_kernel % 2 == 0:
    n_kernel+=1
print(n_kernel)   

dropout_rand = 0.2*torch.rand(1).item()
dropout_p = random.choice([0, dropout_rand])
print(dropout_p)

n_channels = random.choice([16, 32, 64])

#Set model specific parameters
if model_type == 'Linear':
    n_epochs = 600
    n_kernel = 0
    n_showevery_epochs = 100
    dropout_p = 0
    
    #Set the saving and plotting function
    plot = utils.imshowNLinear 
    save = utils.imsaveNLinear 
    
    #Make model
    model_real = Linear(d, data_shape).to(device)
    model_imag = Linear(d, data_shape).to(device)
elif model_type == 'Linear_M':
    n_epochs = 1500 
    n_kernel = 0    
    n_showevery_epochs = 100
    dropout_p = 0
    
    #Set the saving and plotting function
    plot = utils.imshowNLinear
    save = utils.imsaveNLinear

    #Make model
    model_real = Linear_M(d, data_shape).to(device)
    model_imag = Linear_M(d, data_shape).to(device)    
elif model_type == 'Linear_M_Loc':
    n_epochs = 1500 
    n_kernel = 0    
    n_showevery_epochs = 100
    dropout_p = 0
    
    #Set the saving and plotting function
    plot = utils.imshowNLinear
    save = utils.imsaveNLinear

    #Make model
    model_real = Linear_M_Loc(d, data_shape).to(device)
    model_imag = Linear_M_Loc(d, data_shape).to(device)        
elif model_type == 'LinearAE':
    n_epochs = 1000 
    n_showevery_epochs = 100
    n_kernel = 0
    dropout_p = 0
    
    #Set the saving and plotting function
    plot = utils.imshowN
    save = utils.imsaveN

    #Make model
    model_real = LinearAE(d, n_bottleneck).to(device)
    model_imag = LinearAE(d, n_bottleneck).to(device) 
elif model_type == 'LinearAE_M':
    n_epochs = 6000 
    n_showevery_epochs = 1000
    n_kernel = 0
    dropout_p = 0
    
    #Set the saving and plotting function
    plot = utils.imshowN
    save = utils.imsaveN

    #Make model
    model_real = LinearAE_M(d, n_bottleneck).to(device)
    model_imag = LinearAE_M(d, n_bottleneck).to(device)  
elif model_type == 'LinearAE_M_Loc':
    n_epochs = 1500 
    n_showevery_epochs = 100
    n_kernel = 0
    dropout_p = 0
    
    #Set the saving and plotting function
    plot = utils.imshowN
    save = utils.imsaveN

    #Make model
    model_real =  LinearAE_M_Loc(d, n_bottleneck).to(device)
    model_imag = LinearAE_M_Loc(d, n_bottleneck).to(device)           
elif model_type == 'ConvAE':
    n_epochs = 5000
    n_showevery_epochs = 100
    
    #Set the saving and plotting function
    plot = utils.imshowN
    save = utils.imsaveN
    
    #Make model
    model_real = ConvAE().to(device)
    model_imag = ConvAE().to(device)
elif model_type == 'ConvAE_M':
    n_epochs = 3000 
    n_showevery_epochs = 100
    
    #Set the saving and plotting function
    plot = utils.imshowN
    save = utils.imsaveN
    
    #Make model
    model_real = ConvAE_M_shallow(kernel_size=n_kernel, dropout_p=dropout_p).to(device)
    model_imag = ConvAE_M_shallow(kernel_size=n_kernel, dropout_p=dropout_p).to(device)
elif model_type == 'ConvAE_M_Loc':
    n_epochs = 3000
    n_showevery_epochs = 100
    
    #Set the saving and plotting function
    plot = utils.imshowN
    save = utils.imsaveN
    
    #Make model
    model_real = ConvAE_M_Loc(kernel_size=n_kernel, dropout_p=dropout_p).to(device)
    model_imag = ConvAE_M_Loc(kernel_size=n_kernel, dropout_p=dropout_p).to(device)
    
elif model_type == 'unet':
    n_epochs = 3000
    n_showevery_epochs = 100
    n_stride = 1
    
    #Set the saving and plotting function
    plot = utils.imshowN
    save = utils.imsaveN
    
    #Make model
    model_real = UNet(kernel_size=n_kernel, stride=n_stride, dropout_p=dropout_p).to(device)
    model_imag = UNet(kernel_size=n_kernel, stride=n_stride, dropout_p=dropout_p).to(device)
elif model_type == 'unet_M':
    n_epochs = 3000
    n_showevery_epochs = 100
    n_stride = 1
    
    #Set the saving and plotting function
    plot = utils.imshowN
    save = utils.imsaveN
    
    #Make model
    model_real = UNet_M(kernel_size=n_kernel, stride=n_stride, dropout_p=dropout_p).to(device)
    model_imag = UNet_M(kernel_size=n_kernel, stride=n_stride, dropout_p=dropout_p).to(device)
elif model_type == 'UNet_M_LC':
    n_epochs = 3000
    n_showevery_epochs = 100
    n_stride = 1
    
    #Set the saving and plotting function
    plot = utils.imshowN
    save = utils.imsaveN
    
    #Make model
    model_real = UNet_M_LC(kernel_size=n_kernel, stride=n_stride, dropout_p=dropout_p, n_channels_s1 = n_channels).to(device)
    model_imag = UNet_M_LC(kernel_size=n_kernel, stride=n_stride, dropout_p=dropout_p, n_channels_s1 = n_channels).to(device)    
elif model_type == 'UNet_M_Loc':
    n_epochs = 4000
    n_showevery_epochs = 500
    n_stride = 1
    
    #Set the saving and plotting function
    plot = utils.imshowN
    save = utils.imsaveN
    
    #Make model
    model_real = UNet_M_Loc(kernel_size=n_kernel, stride=n_stride, dropout_p=dropout_p, n_channels_s1 = n_channels).to(device)
    model_imag = UNet_M_Loc(kernel_size=n_kernel, stride=n_stride, dropout_p=dropout_p, n_channels_s1 = n_channels).to(device)  
elif model_type == 'UNet_M_Loc_strided':
    n_epochs = 5000
    n_showevery_epochs = 1 #500
    n_stride = 2 
    i_height = data_shape[1]
    i_width = data_shape[2]    
    
    #Set the saving and plotting function
    plot = utils.imshowN
    save = utils.imsaveN
    
    #Make model
    model_real = UNet_M_Loc_strided(kernel_size=n_kernel, stride=n_stride, dropout_p=dropout_p, n_channels_s1 = n_channels, height = i_height, width = i_width).to(device)
    model_imag = UNet_M_Loc_strided(kernel_size=n_kernel, stride=n_stride, dropout_p=dropout_p, n_channels_s1 = n_channels, height = i_height, width = i_width).to(device)       
elif model_type == 'UNet_M_Loc_deepNet':
    if n_subsample > 3:
        raise Exception('Image size not suitable for deepnet')

    #lr = 1e-1
    n_epochs = 4000
    n_showevery_epochs = 500
    n_stride = 1
    
    #Set the saving and plotting function
    plot = utils.imshowN
    save = utils.imsaveN
    
    #Make model
    model_real = UNet_M_Loc_deepNet(kernel_size=n_kernel, stride=n_stride, dropout_p=dropout_p, n_channels_s1 = n_channels).to(device)
    model_imag = UNet_M_Loc_deepNet(kernel_size=n_kernel, stride=n_stride, dropout_p=dropout_p, n_channels_s1 = n_channels).to(device)   
elif model_type == 'wnet':
    lr = 1e-4
    n_epochs = 3000
    n_showevery_epochs = 100
    n_stride = 1
    n_kernel = 5
    
    #Set the saving and plotting function
    plot = utils.imshowN
    save = utils.imsaveN
    
    #Make model
    model_real = WNet(kernel_size=n_kernel, stride=n_stride).to(device)
    model_imag = WNet(kernel_size=n_kernel, stride=n_stride).to(device)    
elif model_type == 'wnet2':
    lr = 1e-4
    n_epochs = 3000
    n_showevery_epochs = 100
    n_stride = 1
    n_kernel = 5
    
    #Set the saving and plotting function
    plot = utils.imshowN
    save = utils.imsaveN
    
    #Make model
    model_real = WNet2(kernel_size=n_kernel, stride=n_stride).to(device)
    model_imag = WNet2(kernel_size=n_kernel, stride=n_stride).to(device)   
else:
    raise Exception('Unknown model type')
    

In [ ]:
#Optimizer
optimizer = torch.optim.SGD(list(model_real.parameters()) + list(model_imag.parameters()), 
                            lr=lr, momentum=momentum, weight_decay=weight_decay)

In [ ]:
#Make scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=100)

In [ ]:
#Train
train_errors   = []
test_accuracys = []
 

lr_prev = lr
for epoch in range(n_epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    t_start = time()
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, MRIs, masks, labels = data[0][0].to(device), data[0][1].to(device), data[0][2].to(device),  data[1].to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs_real = model_real(inputs, MRIs)
        outputs_imag = model_imag(inputs, MRIs)
        outputs = outputs_real + 1j*outputs_imag

        #Mask the border
        outputs = outputs * masks
        labels = labels * masks

        #Calc loss
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Save training loss
        norm_factor_batch = len(trainloader.dataset)/n_batch
        running_loss += loss.item()/norm_factor_batch
    
    #At the end of the epoch, save the train and test errors
    train_error   = running_loss 
    test_accuracy = utils.get_test_accuracy(model_real, model_imag, dataset=testset, criterion=criterion)
    train_errors.append(torch.Tensor([train_error]))
    test_accuracys.append(test_accuracy.view(1,-1))
    if (epoch + 1) % n_showevery_epochs == 0:
        print(f'Epoch: {epoch+1}, Train error = {train_error:.3f}, Test accuracy = {test_accuracy.item():.3f}, Time elapsed = {time()-t_start:.1f}s')
        plot([1-inputs[0], outputs[0], labels[0]], 
                 ['Input', 'Model Output', 'Ground Truth'],n_rows=1)
        save([1-inputs[0], outputs[0], labels[0]], ['training_in', 'training_out', 'training_gt'], 
             tag='_epoch#' + str(epoch+1), path=os.path.join(f_output_images, f_subfolder, f_sub_sub_folder))
        
    #Step the scheduler
    scheduler.step(train_error)
    lr_now = optimizer.state_dict()['param_groups'][0]['lr']
    if lr_now != lr_prev:
        print('Learning Rate =', lr_now, 'at epoch #' + str(epoch+1))
        lr_prev = lr_now

print('Finished Training')
train_errors   = torch.concat(train_errors)
test_accuracys = torch.concat(test_accuracys)

In [ ]:
#Save models
f_model_real = os.path.join(f_model_output, f_subfolder, model_type + '_real.pt')
f_model_imag = os.path.join(f_model_output, f_subfolder, model_type + '_imag.pt')
torch.save(model_real, f_model_real)
torch.save(model_imag, f_model_imag)

#Save errors
f_error_train = os.path.join(f_model_output, f_subfolder, model_type + '_train_error.pt')
f_error_test  = os.path.join(f_model_output, f_subfolder, model_type + '_test_error.pt')
torch.save(train_errors, f_error_train)
torch.save(test_accuracys, f_error_test)

In [ ]:
# save best model
def save_as_best_model():
    #Save models
    f_model_real = os.path.join(f_model_output, f_subfolder, model_type + '_real_best.pt')
    f_model_imag = os.path.join(f_model_output, f_subfolder, model_type + '_imag_best.pt')
    torch.save(model_real, f_model_real)
    torch.save(model_imag, f_model_imag)
    
    #Save errors
    f_error_train = os.path.join(f_model_output, f_subfolder, model_type + '_train_error_best.pt')
    f_error_test  = os.path.join(f_model_output, f_subfolder, model_type + '_test_error_best.pt')
    torch.save(train_errors, f_error_train)
    torch.save(test_accuracys, f_error_test)

if os.path.exists(f_log):
    fid = open(f_log, 'r')
    raw = fid.read()
    fid.close()
    
    best_val = max([float(line.split('val_err: ')[1].split(',')[0]) for line in raw.split('\n')[:-1]])
    if test_accuracys[-1] < best_val:
        #Save current model as best model
        save_as_best_model()
else:
    #Save current model as best model
    save_as_best_model()

In [ ]:
# #Load models
# f_model_real = os.path.join(f_model_output, f_subfolder, model_type + '_real.pt')
# f_model_imag = os.path.join(f_model_output, f_subfolder, model_type + '_imag.pt')
# model_real = torch.load(f_model_real, map_location=torch.device(device))
# model_imag = torch.load(f_model_imag, map_location=torch.device(device))


In [ ]:
plt.figure(figsize=[15,5])

x = np.arange(n_epochs)+1

plt.subplot(1,2,1)
plt.plot(x, train_errors)
plt.title('Training Error')
plt.xlabel('Epoch')
plt.ylabel('MSE')
ymin = train_errors.min()*0.9

if model_type == 'Linear':
    i_epoch = 25
    plt.annotate("epoch " + str(i_epoch), xy=(i_epoch, train_errors[i_epoch-1]), 
                 xytext=(i_epoch+20, train_errors[i_epoch-1]+1000), arrowprops=dict(arrowstyle="->"))
    i_epoch = 300
    plt.annotate("epoch " + str(i_epoch), xy=(i_epoch, train_errors[i_epoch-1]), 
                 xytext=(i_epoch-70, train_errors[i_epoch-1]+2000), arrowprops=dict(arrowstyle="->"))
    i_epoch = 500
    plt.annotate("epoch " + str(i_epoch), xy=(i_epoch, train_errors[i_epoch-1]), 
                 xytext=(i_epoch-100, train_errors[i_epoch-1]+1500), arrowprops=dict(arrowstyle="->"))
else:
    plt.ylim([ymin,train_errors.quantile(.9)*1.1])


plt.subplot(1,2,2)
plt.plot(x, test_accuracys)
plt.title('Validation Error')
plt.xlabel('Epoch')
plt.ylabel('MSE')
ymin = test_accuracys.min()*0.9
if model_type == 'Linear':
    pass
else:
    plt.ylim([ymin,test_accuracys.quantile(.9)*1.1])
plt.savefig(os.path.join(f_output_images, f_subfolder, f_sub_sub_folder, 'errors' + '.png'), pad_inches=0.0)
plt.show()

In [ ]:
#Set to eval
model_real.eval()
model_imag.eval()

real_PC_coeff = 0
real_PC_coeff_val = 0
loss = 0
for i in range(len(testset)):
    with torch.no_grad():
        
        inputs, MRIs, masks, labels = testset[i][0][0].unsqueeze(0).to(device), testset[i][0][1].unsqueeze(0).to(device), testset[i][0][2].unsqueeze(0).to(device), testset[i][1].unsqueeze(0).to(device)
        
        # forward + backward + optimize
        outputs_real = model_real(inputs, MRIs)
        outputs_imag = model_imag(inputs, MRIs)
        outputs = outputs_real + 1j*outputs_imag
        
        #Mask the border
        outputs = outputs * masks
        labels = labels * masks

        # real-valued pearson correlation coefficient
        real_PC_coeff_val = utils.pearson_corr_real(outputs, labels)
        real_PC_coeff += real_PC_coeff_val / len(testset)

        #Calculate loss
        loss += criterion(outputs, labels) / len(testset)
        
        #Difference plot
        with torch.no_grad():
            diff = outputs - labels
            diff_err = diff/labels*100
                
print(real_PC_coeff)
print(loss)

In [ ]:
fid = open(f_log,'a')
fid.write(f'lr: {lr}, seed: {seed}, kernel_size: {n_kernel}, model: {model_type}, val_err: {test_accuracys[-1].item()}, train_err: {train_errors[-1]}, n_epochs: {n_epochs}, dropout_p: {dropout_p}, n_channels: {n_channels}, MRI_loc_impulse_msg: {MRI_loc_impulse_msg}, real_PC_coeff: {real_PC_coeff}  \n')
fid.close()

In [ ]:
#Set to eval
model_real.eval()
model_imag.eval()

# patient 1
NumOfAntz = fdata_pad.shape[0]

def perpatient_val_err(start, stop):
    loss = 0
    real_PC_coeff = 0
    for i in range(start, stop):
        with torch.no_grad():
            inputs, MRIs, masks, labels = testset[i][0][0].unsqueeze(0).to(device), testset[i][0][1].unsqueeze(0).to(device), testset[i][0][2].unsqueeze(0).to(device), testset[i][1].unsqueeze(0).to(device)
            # forward + backward + optimize
            outputs_real = model_real(inputs, MRIs)
            outputs_imag = model_imag(inputs, MRIs)
            outputs = outputs_real + 1j*outputs_imag

            #Mask the border
            outputs = outputs * masks
            labels = labels * masks

            #Calculate loss
            loss += criterion(outputs, labels) / NumOfAntz

            # real-valued pearson correlation coefficient
            real_PC_coeff_val = utils.pearson_corr_complex(outputs, labels)
            real_PC_coeff += real_PC_coeff_val / NumOfAntz

    return loss, real_PC_coeff

val_err_P1, PC_P1 = perpatient_val_err(0, NumOfAntz)
val_err_P2, PC_P2 = perpatient_val_err(NumOfAntz, NumOfAntz * 2 )
val_err_P3, PC_P3 = perpatient_val_err(NumOfAntz * 2, NumOfAntz * 3 )
val_err_P4, PC_P4 = perpatient_val_err(NumOfAntz * 3, NumOfAntz * 4 )
val_err_P5, PC_P5 = perpatient_val_err(NumOfAntz * 4, NumOfAntz * 5 )
val_err_P6, PC_P6 = perpatient_val_err(NumOfAntz * 5, NumOfAntz * 6 )

val_err_all = [val_err_P1.item(), val_err_P2.item(), val_err_P3.item(), val_err_P4.item(), val_err_P5.item(), val_err_P6.item()]
print(val_err_all)
PC_all = [PC_P1.item(), PC_P2.item(), PC_P3.item(), PC_P4.item(), PC_P5.item(), PC_P6.item()]
print(PC_all)

In [ ]:
# real part of complex PC: getting validation error per patient + per antenna
#Set to eval
model_real.eval()
model_imag.eval()


# NumOfAntz = fdata_pad.shape[0]
def perPatient_perAnt_val_err(start, stop):
    loss = []
    real_PC_coeff = []
    for i in range(start, stop):
        with torch.no_grad():
            inputs, MRIs, masks, labels = testset[i][0][0].unsqueeze(0).to(device), testset[i][0][1].unsqueeze(0).to(device), testset[i][0][2].unsqueeze(0).to(device), testset[i][1].unsqueeze(0).to(device)
            # forward + backward + optimize
            outputs_real = model_real(inputs, MRIs)
            outputs_imag = model_imag(inputs, MRIs)
            outputs = outputs_real + 1j*outputs_imag

            #Mask the border
            outputs = outputs * masks
            labels = labels * masks

            #Calculate loss
            loss_rn = criterion(outputs, labels)
            loss.append(torch.tensor([loss_rn]))


            # real-valued pearson correlation coefficient
            real_PC_coeff_val = utils.pearson_corr_complex(outputs, labels)
            real_PC_coeff.append(torch.tensor([real_PC_coeff_val]))

    return loss, real_PC_coeff

val_err_all, PC_all = perPatient_perAnt_val_err(0, NumOfAntz * 6)

val_err_all   = torch.concat(val_err_all)
PC_all = torch.concat(PC_all)


In [ ]:
# plotting different colorbars for OP, GT, and DIFF
plt.set_cmap('viridis')

#Set to eval
model_real.eval()
model_imag.eval()


#for i in range(len(testset)):
i = 1 - 1
print(i)
with torch.no_grad():
    inputs, MRIs, masks, labels = testset[i][0][0].unsqueeze(0).to(device), testset[i][0][1].unsqueeze(0).to(device), testset[i][0][2].unsqueeze(0).to(device), testset[i][1].unsqueeze(0).to(device)

    # forward + backward + optimize
    outputs_real = model_real(inputs, MRIs)
    outputs_imag = model_imag(inputs, MRIs)
    outputs = outputs_real + 1j*outputs_imag

    #Mask the border
    outputs = outputs * masks
    labels = labels * masks

    #Difference plot
    diff = outputs - labels
    rel_err = (outputs.abs() - labels.abs())/(labels.abs()[labels.abs().isnan()==False].max()) * 100
    rel_err_ang = (outputs.angle().abs() - labels.angle().abs())/(labels.angle().abs()[labels.angle().abs().isnan()==False].max()) * 100

    masks2 = masks.clone()
    masks2[masks2==0] = torch.nan
    labels = labels * masks2
    outputs = outputs * masks2
    diff = diff * masks2
    rel_err = rel_err * masks2
    rel_err_ang = rel_err_ang * masks2

    LIST = [labels[0], outputs[0], diff[0], rel_err[0]]
    TITLES = ['Ground Truth', 'Model Output','DIFF', 'Rel_err']

    plot(LIST, TITLES, n_rows=1, colorbar=True, log=True)

    #save(LIST, TITLES, tag='_L' + str(i+1) + '_400MHz', path=os.path.join(f_output_images, f_subfolder, f_sub_sub_folder), figsize_width=17, figsize_height=6, log=True, clim=None)

#    break
LIST = [labels[0], outputs[0], diff[0], rel_err[0]]
n_cols = 4

max_plots = torch.zeros(n_cols)
min_plots = torch.zeros(n_cols)

for ind in range(len(LIST)):
    max_plots[ind] = LIST[ind].abs()[LIST[ind].abs().isnan() == False].max()
    min_plots[ind] = LIST[ind].abs()[LIST[ind].abs().isnan() == False].min()

max_plt = torch.log(max_plots[1])
min_plt = torch.log(min_plots[1])

# plt.jet()

In [ ]:
# PAPERs+relative error: plotting same colorbars for GT, OP, and different for Diff 
LIST = [labels[0], outputs[0], diff[0], rel_err[0]]
n_cols = len(LIST)
log = True

max_plots = torch.zeros(n_cols)
min_plots = torch.zeros(n_cols)

for ind in range(len(LIST)):
    max_plots[ind] = LIST[ind].abs()[LIST[ind].abs().isnan() == False].max()
    min_plots[ind] = LIST[ind].abs()[LIST[ind].abs().isnan() == False].min()

# print(max_plots)
# print(min_plots)

if log == True:
    max_a = max_plots[0].max().log()
    min_a = min_plots[0].min().log()
else:
    max_a = max_plots[0].max()
    min_a = min_plots[0].min()

max_b = max_plots[0]
min_b = min_plots[0]

max_c = max_plots[-1]
min_c = min_plots[-1]

# print(max_b)
# print(min_b)

f, ax = plt.subplots(1, 4, figsize=(14, 3.5))
if log == True:
    l0 = ax[0].imshow(LIST[0].cpu().abs().log(), vmin=min_a, vmax=max_a)
    ax[0].invert_yaxis()
    ax[0].axis('off')
    l1 = ax[1].imshow(LIST[1].cpu().abs().log(), vmin=min_a, vmax=max_a)
    ax[1].invert_yaxis()
    ax[1].axis('off')
    l2 = ax[2].imshow(LIST[2].cpu().abs(), vmin=min_b, vmax=max_b)
    ax[2].invert_yaxis()
    ax[2].axis('off')
    l3 = ax[3].imshow(LIST[3].cpu().abs(), vmin=min_c, vmax=max_c)
    ax[3].invert_yaxis()
    ax[3].axis('off')
else:
    l0 = ax[0].imshow(LIST[0].cpu().abs(), vmin=min_a, vmax=max_a)
    ax[0].invert_yaxis()
    ax[0].axis('off')
    l1 = ax[1].imshow(LIST[1].cpu().abs(), vmin=min_a, vmax=max_a)
    ax[1].invert_yaxis()
    ax[1].axis('off')
    l2 = ax[2].imshow(LIST[2].cpu().abs(), vmin=min_b, vmax=max_b)
    ax[2].invert_yaxis()
    ax[2].axis('off')
    # l3 = ax[3].imshow(LIST[3].cpu().abs(), vmin=min_b, vmax=max_b)
    # ax[3].invert_yaxis()
    # ax[3].axis('off')


plt.subplots_adjust(wspace=0, hspace=0)
f.colorbar(l0, ax=list(ax[0:2]), orientation='vertical', pad=0.03, shrink=0.85)
f.colorbar(l2, ax=ax[2], orientation='vertical', pad=0.03, shrink=0.85)
f.colorbar(l3, ax=ax[3], orientation='vertical', pad=0.03, shrink=0.85)
plt.set_cmap('viridis')

#plt.jet()

plt.show()

path=os.path.join(f_output_images, f_subfolder, f_sub_sub_folder)
f.savefig(os.path.join(path, freq + '_' + str(i+1) +'_' + model_type + '.png'), dpi=150, bbox_inches='tight')


In [ ]:
###new with 2 colorbars
#new: (0, 2pi) angle range: plotting ANGLES: same colorbars for GT, OP, and  Diff 
plt.set_cmap('twilight_shifted')

LIST = [labels[0], outputs[0], diff[0], rel_err_ang[0]]
n_cols = len(LIST)
log = False # log will always be set to false for angles

max_a = 2*torch.pi
min_a = 0

max_b = LIST[3].abs()[LIST[3].abs().isnan() == False].max()
min_b = LIST[3].abs()[LIST[3].abs().isnan() == False].min()

f, ax = plt.subplots(1, 4, figsize=(14, 3.5))
# plt.jet()

pi_def = torch.tensor(torch.pi);

l0 = ax[0].imshow(torch.remainder(LIST[0].cpu().angle(),2*pi_def), vmin=min_a, vmax=max_a, interpolation="nearest")
ax[0].invert_yaxis()
ax[0].axis('off')
l1 = ax[1].imshow(torch.remainder(LIST[1].cpu().angle(),2*pi_def), vmin=min_a, vmax=max_a, interpolation="nearest")
ax[1].invert_yaxis()
ax[1].axis('off')
metric_b = (torch.remainder(LIST[0].cpu().angle(),2*pi_def) - torch.remainder(LIST[1].cpu().angle(),2*pi_def)).abs()
angle_dist = torch.minimum(metric_b, 2*torch.pi - metric_b)
l2 = ax[2].imshow(angle_dist, vmin=min_a, vmax=max_a)
ax[2].invert_yaxis()
ax[2].axis('off')
#l3 = ax[3].imshow(LIST[3].cpu(), vmin=min_b, vmax=max_b)
ax[3].invert_yaxis()
ax[3].axis('off')


plt.subplots_adjust(wspace=0, hspace=0)
f.colorbar(l0, ax=list(ax[0:2]), orientation='vertical', pad=0.03, shrink=0.85)
f.colorbar(l2, ax=ax[2], orientation='vertical', pad=0.03, shrink=0.85)
# f.colorbar(l3, ax=ax[3], orientation='vertical', pad=0.03, shrink=0.6)
plt.set_cmap('twilight_shifted')

plt.show()

path=os.path.join(f_output_images, f_subfolder, f_sub_sub_folder)
f.savefig(os.path.join(path, 'angles'+ freq + '_' + str(i+1) +'_' +  model_type + '.png'), dpi=150, bbox_inches='tight')
